In [ ]:
# This notebook is referenced from the youtube tutorial 'Pretrained CNN (VGG16 - imagenet) features for semantic segmentation using Random Forest
# Reference: https://www.youtube.com/watch?v=vgdFovAZUzM

In [8]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle

from keras.models import Sequential, Model
from keras.layers import Conv2D
import os
from keras.applications.vgg16 import VGG16


In [12]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 1024 #Resize images (height  = X, width = Y)
SIZE_Y = 996

In [13]:
#Capture training image info as a list
train_images = []

In [14]:
for directory_path in glob.glob("/home/dsp_tameo/tamiyo-p3/models/pre-trained-cnn/data/train/"):
    for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        #train_labels.append(label)

In [15]:
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

In [16]:
#Capture mask/label info as a list
train_masks = [] 
for directory_path in glob.glob("/home/dsp_tameo/tamiyo-p3/models/pre-trained-cnn/data/masks/"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.tif")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        #train_labels.append(label)
        

In [17]:
#Convert list to array for machine learning processing   
train_masks = np.array(train_masks)

In [18]:
#Use customary x_train and y_train variables
X_train = train_images
y_train = train_masks

In [19]:
#Load VGG16 model wothout classifier/fully connected layers
#Load imagenet weights that we are going to use as feature generators
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE_X, SIZE_Y, 3))

In [20]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
 

In [21]:
   
VGG_model.summary()  #Trainable parameters will be 0

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1024, 996, 3)]    0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 1024, 996, 64)     1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 1024, 996, 64)     36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 512, 498, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 512, 498, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 512, 498, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 256, 249, 128)     0     

In [24]:

#New model with only first 2 conv layers
new_model = Model(inputs=VGG_model.input, outputs=VGG_model.get_layer('block1_conv2').output)
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1024, 996, 3)]    0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 1024, 996, 64)     1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 1024, 996, 64)     36928     
Total params: 38,720
Trainable params: 0
Non-trainable params: 38,720
_________________________________________________________________


In [27]:
X_train

array([], dtype=float64)

In [1]:
# #Now, let us apply feature extractor to our training data
features=new_model.predict(X_train)

In [2]:
# #Plot features to view them
square = 8
ix=1
for _ in range(square):
    for _ in range(square):
        ax = plt.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(features[0,:,:,ix-1], cmap='gray')
        ix +=1
plt.show()

In [ ]:
#Reassign 'features' as X to make it easy to follow
X=features
X = X.reshape(-1, X.shape[3])  #Make it compatible for Random Forest and match Y labels

In [ ]:
#Reshape Y to match X
Y = y_train.reshape(-1)

In [ ]:
#Combine X and Y into a dataframe to make it easy to drop all rows with Y values 0
#In our labels Y values 0 = unlabeled pixels. 
dataset = pd.DataFrame(X)
dataset['Label'] = Y
print(dataset['Label'].unique())
print(dataset['Label'].value_counts())

In [ ]:
##If we do not want to include pixels with value 0 
##e.g. Sometimes unlabeled pixels may be given a value 0.
dataset = dataset[dataset['Label'] != 0]

In [ ]:
#Redefine X and Y for Random Forest
X_for_RF = dataset.drop(labels = ['Label'], axis=1)
Y_for_RF = dataset['Label']

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
model.fit(X_for_RF, Y_for_RF) 

In [ ]:
#Save model for future use
filename = 'RF_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
#Load model.... 
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#Test on a different image
#READ EXTERNAL IMAGE...
test_img = cv2.imread('/home/dsp_tameo/tamiyo-p3/models/pre-trained-cnn/data/train/5da0713f251f0558fda439e41e3f178b1ce81f17ba669e7652e904676ef382e3.tif', cv2.IMREAD_COLOR)       
test_img = cv2.resize(test_img, (SIZE_Y, SIZE_X))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img, axis=0)

In [ ]:

X_test_feature = new_model.predict(test_img)
X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])

prediction = loaded_model.predict(X_test_feature)

In [ ]:
#View and Save segmented image
prediction_image = prediction.reshape(mask.shape)
plt.imshow(prediction_image, cmap='gray')
plt.imsave('/home/dsp_tameo/tamiyo-p3/models/pre-trained-cnn/data/results/5da0713f251f0558fda439e41e3f178b1ce81f17ba669e7652e904676ef382e3.png', prediction_image, cmap='gray')